In [1]:
!pip install -U langchain-community

In [2]:
!pip install -q langchain torch transformers sentence-transformers datasets faiss-cpu

In [3]:
!pip install -Uq datasets

In [4]:
from datasets import load_dataset
from langchain.docstore.document import Document

dataset_name = "databricks/databricks-dolly-15k"
page_content_column = "context"

# Load the dataset
dataset = load_dataset(dataset_name)

# Manually create Document objects
documents = []
for item in dataset['train']:
    # Assuming 'context' is the content and other keys can be metadata
    page_content = item.get(page_content_column, "")
    metadata = {key: value for key, value in item.items() if key != page_content_column}
    documents.append(Document(page_content=page_content, metadata=metadata))

# The 'documents' variable now contains the loaded data in the correct format.
# You can proceed with text splitting and embedding as before.

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)

docs = text_splitter.split_documents(documents)

print(docs[0])


page_content='Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia's domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney.' metadata={'instruction': 'When did Virgin Australia start operating?', 'response': 'Virgin Australia commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route.', 'category': 'closed_qa'}


In [7]:
import torch
print(torch.cuda.is_available())

True


In [8]:
from langchain.embeddings import HuggingFaceEmbeddings

modelPath = "sentence-transformers/all-MiniLM-l6-v2"
model_kwargs = {'device':'cuda'}
encode_kwargs = {'normalize_embeddings': False}

# Initialisation de HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

# (Optionnel) Tester la création d’un embedding
text = "This is a test document."
query_result = embeddings.embed_query(text)

# Afficher les 3 premières dimensions du vecteur
print(query_result[:3])

/tmp/ipython-input-8-1747689788.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


[-0.0383385568857193, 0.12346474081277847, -0.02864294871687889]


In [9]:
from langchain.vectorstores import FAISS

# Création de l'index FAISS à partir des documents et des embeddings
db = FAISS.from_documents(docs, embeddings)

In [13]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain.llms import HuggingFacePipeline

# Modèle de génération (plus adapté à LangChain)
model_name = "google/flan-t5-base"  # léger, CPU-friendly

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Crée un pipeline text2text (pas question-answering)
hf_pipeline = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=512
)

# Intègre à LangChain
llm = HuggingFacePipeline(pipeline=hf_pipeline)


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cuda:0


In [14]:
from langchain.chains import RetrievalQA

retriever = db.as_retriever(search_kwargs={"k": 4})

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="refine",  # Changed from "refine" to "stuff"
    retriever=retriever,
    return_source_documents=False  # passe à True si tu veux afficher les documents utilisés
)

In [23]:
query = "Buckwheat pancake recipe?"

answer = qa.run(query)

print("Answer:", answer)

Answer: The Hot Brown is an open-faced sandwich of turkey breast and ham and bacon, covered in creamy Mornay sauce and baked or broiled until the bread is crisp and the sauce begins to brown. Cheddar cheese or American cheese may be added for the sauce. Alternatives for garnishes include tomatoes, mushroom slices, and, very rarely, canned peaches. A Hot Brown sandwich (sometimes known as a Louisville Hot Brown or Kentucky Hot Brown) is an American hot sandwich originally created at the Brown Hotel in Louisville, Kentucky, by Fred K. Schmidt in 1926. It is a variation of traditional Welsh rarebit and was one of two signature sandwiches created by chefs at the Brown Hotel shortly after its founding in 1923. It was created to serve as an alternative to ham and egg late-night dinners. Ingredients The Hot Brown is an open-faced sandwich of turkey breast and ham and bacon, covered in creamy Mornay sauce and baked or broiled until the bread is crisp and the sauce begins to brown. Cheddar chee